In [4]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")

Environment Variables are set OK


In [6]:
#Example Use case -> character description generator for fantasy RPG
#Takes a random dictionary of character traits and generates a written character description

races=["elf","human","dwarf","halfling","gnome"]
complections=["pale","dark","fair","ruddy","swarthy"]
heights=["tall","short","average"]
builds=["slender","stocky","muscular","portly"]
features=["sharp","soft","angular","round"]
eyes=["bright","dark","piercing","dull"]
hairs=["long","short","curly","straight","wavy"]
clothes=["fine","simple","elegant","gaudy","tattered"]
sophistication=["refined","crude","simple","complex"]
mannerisms=["graceful","clumsy","boisterous","quiet"]
virtues=["honest","loyal","brave","generous","kind"]


import random

#Come up with a random dictionary of character traits
def GenerateCharacterDescDictionary():
    traits={}
    race=random.choice(races)
    complection=random.choice(complections)
    height=random.choice(heights)
    build=random.choice(builds)
    feature=random.choice(features)
    eye=random.choice(eyes)
    hair=random.choice(hairs)
    clothe=random.choice(clothes)
    sophisticate=random.choice(sophistication)
    mannerism=random.choice(mannerisms)
    virtue=random.choice(virtues)
    traits={"race":race,"complection":complection,"height":height,"build":build,"feature":feature,"eye":eye,"hair":hair,"clothes":clothe,"sophistication":sophisticate,"mannerism":mannerism,"virtue":virtue}
    return traits

#Turn the dictionary into a line separated string
def GenerateCharacterDescList(traits):
    desc="They have the followng traits:\n"
    for key in traits:
        desc+=key+": "+traits[key]+"\n"
    return desc
    
            
    
#Test code
traits=GenerateCharacterDescDictionary()
print(GenerateCharacterDescList(traits))


They have the followng traits:
race: halfling
complection: ruddy
height: short
build: muscular
feature: sharp
eye: bright
hair: long
clothes: elegant
sophistication: crude
mannerism: boisterous
virtue: loyal



In [7]:
#Produce sample character descriptions
totalRecords=10
requestTexts=[]
for i in range(totalRecords):
    traits=GenerateCharacterDescDictionary()
    requestTexts.append(GenerateCharacterDescList(traits))
requestTexts

['They have the followng traits:\nrace: dwarf\ncomplection: pale\nheight: tall\nbuild: slender\nfeature: angular\neye: bright\nhair: curly\nclothes: tattered\nsophistication: simple\nmannerism: clumsy\nvirtue: loyal\n',
 'They have the followng traits:\nrace: human\ncomplection: swarthy\nheight: average\nbuild: slender\nfeature: sharp\neye: piercing\nhair: wavy\nclothes: tattered\nsophistication: complex\nmannerism: boisterous\nvirtue: honest\n',
 'They have the followng traits:\nrace: elf\ncomplection: dark\nheight: tall\nbuild: stocky\nfeature: sharp\neye: piercing\nhair: curly\nclothes: elegant\nsophistication: crude\nmannerism: graceful\nvirtue: loyal\n',
 'They have the followng traits:\nrace: human\ncomplection: pale\nheight: tall\nbuild: slender\nfeature: round\neye: piercing\nhair: wavy\nclothes: gaudy\nsophistication: complex\nmannerism: clumsy\nvirtue: honest\n',
 'They have the followng traits:\nrace: elf\ncomplection: ruddy\nheight: average\nbuild: slender\nfeature: round\n

In [8]:
#Main Function
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str,systemMessage:str,query:str):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)

#Post to a service bus queue    
def AzurePost_ServiceBus(connection_string: str, queue_name: str, json_payload):
    with ServiceBusClient.from_connection_string(connection_string) as client:
        print(json_payload["Model"])
        sender = client.get_queue_sender(queue_name=queue_name)
        message = ServiceBusMessage(json.dumps(json_payload))
        with sender:
            sender.send_messages(message)
            print(f"Queued message: {json_payload['UUID']}")

#Take a list of queries and turn them into results
def BatchTransForm(Model:str,systemMessage:str,queries:List[str],DynamicUI:bool=True,WebUI:bool=False,ShowWebUI:bool=False)->List[str]:
    
    #Put every prompt into the LLM_Requests pool
    LLMrequests=[]
    for query in queries:
        request=LLMRequest(Model,systemMessage,query)
        LLMrequests.append(request)
        AzurePost_ServiceBus(EndPoint_Queries,"llm-queries",request.to_json())
    
    #Monitor the results queue
    
    
    #Output the results
    
#Test code
systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""
BatchTransForm("llama3.1",systemPrompt,requestTexts)


llama3.1
Queued message: bc9a3ee3-d3e6-43af-9374-446fc1abc40d
llama3.1
Queued message: bf3b0b15-2ec9-404e-a331-49b346b991d4
llama3.1
Queued message: 5dee14c4-80ec-4ff0-8103-d00e44956049
llama3.1
Queued message: 70459dd7-9c9e-4dff-b2db-861e70d9d42b
llama3.1
Queued message: 1c94c2fc-a884-4291-a30c-67635399005c
llama3.1
Queued message: 6b577115-3601-451d-be24-f06b0a5e17cc
llama3.1
Queued message: 44dd71a2-51a0-44e3-821e-e16369b6dd1e
llama3.1
Queued message: 5980331b-2dc3-49b2-9b6a-fee4e25eba8c
llama3.1
Queued message: 16b5613f-c956-4435-8971-fc0066377b4d
llama3.1
Queued message: a5523706-744e-4f65-a625-a7c2b7bbb060
